In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [4]:
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test )

In [7]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [8]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [19]:
class LogisticRegression(nn.Module):

  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = LogisticRegression(n_features)

In [20]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)


In [21]:
num_epochs = 100
for epoch in range(num_epochs):
  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)

  loss.backward()

  optimizer.step()
  optimizer.zero_grad()

  if (epoch + 1) % 10 == 0:
    print(f'epoch: {epoch + 1}, loss = {loss.item():.4f}')



epoch: 10, loss = 0.5801
epoch: 20, loss = 0.4696
epoch: 30, loss = 0.4006
epoch: 40, loss = 0.3539
epoch: 50, loss = 0.3201
epoch: 60, loss = 0.2946
epoch: 70, loss = 0.2745
epoch: 80, loss = 0.2583
epoch: 90, loss = 0.2448
epoch: 100, loss = 0.2335


In [22]:
with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  print(f'accuracy = {acc:.4f}')

accuracy = 0.9123
